In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d ashkhagan/figshare-brain-tumor-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:
! unzip figshare-brain-tumor-dataset.zip

unzip:  cannot find or open figshare-brain-tumor-dataset.zip, figshare-brain-tumor-dataset.zip.zip or figshare-brain-tumor-dataset.zip.ZIP.


In [4]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input,Concatenate
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
import cv2
from tensorflow import keras

In [5]:
datadir="/content/dataset/data"
N=3064
traindata=[]

for i in range(1,N+1):
    filename=str(i)+".mat"
    data=h5py.File(os.path.join(datadir,filename),"r")
    traindata.append(data)

    if i%100==0:
      print(filename)

OSError: ignored

In [ ]:
import random

random.shuffle(traindata)

In [ ]:
train_images = []
train_labels = []
test_images = []
test_labels = []
temp=round(4*3064/5)

for i in range(temp):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    train_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    train_labels.append(label)

for i in range(temp,3064):
  img = traindata[i]["cjdata"]["image"]
  if img.shape==(512,512):
    img=np.expand_dims(img,axis=2)
    test_images.append(img)
    lbl = traindata[i]["cjdata"]["label"]
    label=int(lbl[0])-1
    test_labels.append(label) 

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
image_size = [512,512]

In [ ]:
img_input = Input(shape=(512,512,1))
img_conc = Concatenate()([img_input, img_input, img_input])  

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor = img_conc)

for layer in base_model.layers:
    layer.trainable = False
    
X = base_model.output
X = layers.Flatten()(X)

X = layers.Dense(512, kernel_initializer='he_uniform')(X)
X = layers.Dropout(0.4)(X)
X = layers.BatchNormalization()(X)
X = Activation('relu')(X)

X = layers.Dense(128, kernel_initializer='he_uniform')(X)
X = layers.Dropout(0.4)(X)
X = layers.BatchNormalization()(X)
X = Activation('relu')(X)

X = layers.Dense(16, kernel_initializer='he_uniform')(X)
X = layers.Dropout(0.4)(X)
X = layers.BatchNormalization()(X)
X = Activation('relu')(X)


output = layers.Dense(3, activation='softmax')(X)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
r = model.fit(train_images,train_labels,epochs=10,batch_size=64,shuffle = False)

In [ ]:
result=model.evaluate(test_images,test_labels)
print(result)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss_VGGnet')

In [ ]:
# plot the loss
plt.plot(r.history['accuracy'], label='train accuracy')
# plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(model, legend=True,draw_volume=True)

In [ ]:
label_pred_keras = model.predict(test_images)
print(label_pred_keras)

In [ ]:
label_pred_keras = model.predict(test_images[60][None,...])
print(label_pred_keras)

In [ ]:
final_class = np.unravel_index(label_pred_keras.argmax(),label_pred_keras.shape)
if final_class[1] == 0:
  print("meningioma")
elif final_class[1] == 1:
  print("glioma")
else:
  print("pituitary")

In [ ]:
model.save('ResNet50_tranferr.h5')

In [ ]:
!cp "/content/ResNet50_tranferr.h5" "/content/drive/MyDrive/Final_year_project/Pri/"